<a href="https://colab.research.google.com/github/sharkawy98/data_analysis/blob/main/moviesPreprocessing_and_simpleRecs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from nltk.stem.snowball import SnowballStemmer

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/datasets/movies/tmdb_5000_movies.csv')
credits = pd.read_csv('/content/drive/MyDrive/datasets/movies/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
# merge the two files
data = movies.merge(credits, left_on='id', right_on='movie_id')
data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,movie_id,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [ ]:
data.isnull().sum()  # check count of null values

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title_x                    0
vote_average               0
vote_count                 0
movie_id                   0
title_y                    0
cast                       0
crew                       0
dtype: int64

In [ ]:
# Change JSON string to Python objects
data['genres'] = data['genres'].apply(literal_eval)
data['keywords'] = data['keywords'].apply(literal_eval) 
data['production_companies'] = data['production_companies'].apply(literal_eval) 
data['cast'] = data['cast'].apply(literal_eval)
data['crew'] = data['crew'].apply(literal_eval)

In [ ]:
def get_director(crew):
    '''Return director of each movie if exist, else return NaN'''
    for m in crew:
        if m['job'] == 'Director':
            return m['name']
    return np.nan

In [ ]:
data['director'] = data['crew'].apply(get_director)  # get director name

data['cast'] = data['cast'].apply(lambda cast: [m['name'] for m in cast])  # get list of cast names

data['keywords'] = data['keywords'].apply(lambda keywords: [k['name'] for k in keywords])  # get list of keywords describing movie

In [ ]:
# change names to lower case and remove spaces
# for top 3 actors
data['cast'] = data['cast'].apply(lambda cast: [str.lower(name.replace(" ", "")) for name in cast[:3]])

In [ ]:
# change names to lower case and remove spaces
# for each director
# and multilpy by 3 to give higher weight
data['director'] = data['director'].astype('str').apply(lambda name: [str.lower(name.replace(" ", ""))] * 3)

In [ ]:
# get all keywords occurrences in a pandas.Series
s = data.apply(lambda x: pd.Series(x['keywords']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


0       culture clash
0              future
0           space war
0        space colony
0             society
            ...      
4800    postal worker
4802        obsession
4802        camcorder
4802            crush
4802       dream girl
Name: keyword, Length: 36194, dtype: object

In [ ]:
# count occurrence of each keyword 
s = s.value_counts()
s

woman director          324
independent film        318
duringcreditsstinger    307
based on novel          197
murder                  189
                       ... 
obese                     1
paraguay                  1
orléans                   1
hostage negotiator        1
grim reaper               1
Name: keyword, Length: 9813, dtype: int64

In [ ]:
s = s[s > 1]  # remove keywords with only one occurrence 

In [ ]:
def filter_keywords(keywords):
    '''filter each movie keywords from ones occurred once'''
    words = []
    for k in keywords: 
        if k in s:
            words.append(k)
    return words

stemmer = SnowballStemmer('english')

In [ ]:
# filter keywords & change each one to to lower case and remove spaces
# Also apply stemming to each keyword
data['keywords'] = data['keywords'].apply(filter_keywords)  
data['keywords'] = data['keywords'].apply(lambda keywords: [str.lower((stemmer.stem(k)).replace(' ', '')) for k in keywords])

In [ ]:
# get list of production companies
data['production_companies'] = data['production_companies'].apply(lambda companies: [str.lower(c['name'].replace(' ', '')) for c in companies])

# get list of movie genres
data['genres'] = data['genres'].apply(lambda genres: [str.lower(g['name'].replace(' ', '')) for g in genres])  # get list of movie's genres

In [ ]:
# combine all interesting columns to make feature used to make recommendations 
data['features'] = data['keywords'] + data['cast'] + data['genres'] + data['production_companies'] + data['director']
data['features'] = data['features'].apply(lambda t: ' '.join(t))
data.features

0       cultureclash futur spacewar spacecoloni societ...
1       ocean drugabus exoticisland eastindiatradingco...
2       spi basedonnovel secretag sequel mi6 britishse...
3       dccomic crimefight terrorist secretident burgl...
4       basedonnovel mar medallion spacetravel princes...
                              ...                        
4798    unitedstates–mexicobarri leg carlosgallardo ja...
4799    edwardburns kerrybishé marshadietlein comedy r...
4800    date loveatfirstsight narrat investig team eri...
4801    danielhenney elizacoupe billpaxton danielhsia ...
4802    obsess camcord crush dreamgirl drewbarrymore b...
Name: features, Length: 4803, dtype: object

---
---

# Recommendations

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, sigmoid_kernel, linear_kernel
import scipy.sparse as sp

In [ ]:
# tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
# tfidf_matrix = tf.fit_transform(data['overview'])
# tfidf_matrix.shape
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
# count_matrix = count.fit_transform(data['features'])
# count_matrix.shape
# cosine_sim = cosine_similarity(count_matrix, count_matrix)

# combine_sparse = sp.hstack([tfidf_matrix, count_matrix], format='csr')
# cosine_sim = cosine_similarity(combine_sparse, combine_sparse)

In [ ]:
def load_data():
    data = pd.read_csv('/content/drive/MyDrive/datasets/movies/movies.csv')
    data = data[['movie_id', 'title_x', 'features']]
    return data

def get_similarities(data):
    count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
    count_matrix = count.fit_transform(data['features'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    return cosine_sim

def recommended_movies(title, data, similarities):
    indices = pd.Series(data.index, index=data['title_x'].str.lower())

    movie_idx = indices[title]
    sim_scores = list(enumerate(similarities[movie_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    recommendations_indices = [i[0] for i in sim_scores]
    
    return data.iloc[recommendations_indices][['movie_id', 'title_x']]

In [ ]:
def get_recommendations(movie_name):
        '''Return the top 30 recommended movies'''
        movie_name = movie_name.lower()

        movies = data
        # movies = load_data()
        similarities = get_similarities(data)

        if movie_name not in (movies['title_x'].str.lower()).values:
                return 'Movie not found !'

        else:
                recommendations = recommended_movies(movie_name, data, similarities)
                recommendations.columns = ['tmdb_id', 'title']
                return recommendations.to_dict('records')

In [ ]:
get_recommendations('the godfather')

[{'title': 'The Godfather: Part II', 'tmdb_id': 240},
 {'title': 'The Rainmaker', 'tmdb_id': 11975},
 {'title': 'The Godfather: Part III', 'tmdb_id': 242},
 {'title': 'The Cotton Club', 'tmdb_id': 2148},
 {'title': 'The Conversation', 'tmdb_id': 592},
 {'title': 'Peggy Sue Got Married', 'tmdb_id': 10013},
 {'title': 'The Outsiders', 'tmdb_id': 227},
 {'title': 'Twixt', 'tmdb_id': 78381},
 {'title': 'Apocalypse Now', 'tmdb_id': 28},
 {'title': 'New York Stories', 'tmdb_id': 9686},
 {'title': 'Dracula', 'tmdb_id': 6114},
 {'title': 'Payback', 'tmdb_id': 2112},
 {'title': '10th & Wolf', 'tmdb_id': 13197},
 {'title': 'Collateral', 'tmdb_id': 1538},
 {'title': 'GoodFellas', 'tmdb_id': 769},
 {'title': 'Blood Done Sign My Name', 'tmdb_id': 41894},
 {'title': 'Kiss the Girls', 'tmdb_id': 9437},
 {'title': 'A Simple Plan', 'tmdb_id': 10223},
 {'title': 'Patriot Games', 'tmdb_id': 9869},
 {'title': 'Black Water Transit', 'tmdb_id': 310706},
 {'title': 'The Shawshank Redemption', 'tmdb_id': 278}